## APTyzer

In order to run the APTyzer select the cell below and press the run button above or  `shift` +  `enter`:

In [5]:
%reset -f
from tkinter import *
from tkinter.ttk import Combobox
from tkinter import messagebox
from tkinter import filedialog
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
import numpy as np
import math
from matplotlib import gridspec
import re
from tkinter.filedialog import asksaveasfile
import tkinter.scrolledtext as tkst
import webbrowser
from mpl_toolkits.mplot3d import proj3d
import pandas as pd
import h5py
import struct
from ase.data import chemical_symbols
import os


def read_pos(data):
    """ Loads an APT .pos file as a pandas dataframe.

    Columns:
        x: Reconstructed x position
        y: Reconstructed y position
        z: Reconstructed z position
        Da: mass/charge ratio of ion"""
    # read in the data
    n = int(len(data)/4)
    d = struct.unpack('>'+'f'*n,data)
                    # '>' denotes 'big-endian' byte order
    # unpack data
    pos = pd.DataFrame({'x': d[0::4],
                        'y': d[1::4],
                        'z': d[2::4],
                        'Da': d[3::4]})
    return pos


def read_epos(data):
    """Loads an APT .epos file as a pandas dataframe.

    Columns:
        x: Reconstructed x position
        y: Reconstructed y position
        z: Reconstructed z position
        Da: Mass/charge ratio of ion
        ns: Ion Time Of Flight
        DC_kV: Potential
        pulse_kV: Size of voltage pulse (voltage pulsing mode only)
        det_x: Detector x position
        det_y: Detector y position
        pslep: Pulses since last event pulse (i.e. ionisation rate)
        ipp: Ions per pulse (multihits)

     [x,y,z,Da,ns,DC_kV,pulse_kV,det_x,det_y,pslep,ipp].
        pslep = pulses since last event pulse
        ipp = ions per pulse

    When more than one ion is recorded for a given pulse, only the
    first event will have an entry in the "Pulses since last evenT
    pulse" column. Each subsequent event for that pulse will have
    an entry of zero because no additional pulser firings occurred
    before that event was recorded. Likewise, the "Ions Per Pulse"
    column will contain the total number of recorded ion events for
    a given pulse. This is normally one, but for a sequence of records
    a pulse with multiply recorded ions, the first ion record will
    have the total number of ions measured in that pulse, while the
    remaining records for that pulse will have 0 for the Ions Per
    Pulse value.
        ~ Appendix A of 'Atom Probe tomography: A Users Guide',
          notes on ePOS format."""
    # read in the data
    n = int(len(data)/4)
    rs = n / 11
    d = struct.unpack('>'+'fffffffffII'*int(rs),data)
                    # '>' denotes 'big-endian' byte order
    # unpack data
    pos = pd.DataFrame({'x': d[0::11],
                        'y': d[1::11],
                        'z': d[2::11],
                        'Da': d[3::11],
                        'ns': d[4::11],
                        'DC_kV': d[5::11],
                        'pulse_kV': d[6::11],
                        'det_x': d[7::11],
                        'det_y': d[8::11],
                        'pslep': d[9::11], # pulses since last event pulse
                        'ipp': d[10::11]}) # ions per pulse
    return pos


def read_rrng(f):
    """Loads a .rrng file produced by IVAS. Returns two dataframes of 'ions'
    and 'ranges'."""
    import re

    rf = open(f,'r').readlines()

    patterns = re.compile(r'Ion([0-9]+)=([A-Za-z0-9]+).*|Range([0-9]+)=(\d+.\d+) +(\d+.\d+) +Vol:(\d+.\d+) +([A-Za-z:0-9 ]+) +Color:([A-Z0-9]{6})')
    patterns2 = re.compile(r'ion([0-9]+)=([A-Za-z0-9]+).*|range([0-9]+)=(\d+.\d+) +(\d+.\d+) +vol:(\d+.\d+) +([A-Za-z:0-9 ]+) +color:([A-Z0-9]{6})')
    ions = []
    rrngs = []
    for line in rf:
        m = patterns.search(line)
        m2 = patterns2.search(line)
        if m:
            if m.groups()[0] is not None:
                ions.append(m.groups()[:2])
            else:
                rrngs.append(m.groups()[2:])
        if m2:
            if m2.groups()[0] is not None:
                ions.append(m2.groups()[:2])
            else:
                rrngs.append(m2.groups()[2:])
    ions = pd.DataFrame(ions, columns=['number','name'])
    ions.set_index('number',inplace=True)
    rrngs = pd.DataFrame(rrngs, columns=['number','lower','upper','vol','comp','colour'])
    rrngs.set_index('number',inplace=True)

    rrngs[['lower','upper','vol']] = rrngs[['lower','upper','vol']].astype(float)
    rrngs[['comp','colour']] = rrngs[['comp','colour']].astype(str)

    return ions,rrngs

def label_ions(pos,rrngs):
    """labels ions in a .pos or .epos dataframe (anything with a 'Da' column)
    with composition and colour, based on an imported .rrng file."""
    #rl=list(rrngs)
    #rd=pd.DataFrame(rl.pop(1),columns=['lower','upper','vol','comp','colour'])
    rd=pd.DataFrame(rrngs,columns=['lower','upper','vol','comp','colour'])
    pos['comp'] = ''
    pos['colour'] = '#FFFFFF'

    for n,r in rd.iterrows():
        pos.loc[(pos.Da >= r.lower) & (pos.Da <= r.upper),['comp','colour']] = [r['comp'],'#' + r['colour']]

    return pos


def deconvolve(lpos):
    """Takes a composition-labelled pos file, and deconvolves
    the complex ions. Produces a dataframe of the same input format
    with the extra columns:
       'element': element name
       'n': stoichiometry
    For complex ions, the location of the different components is not
    altered - i.e. xyz position will be the same for several elements."""

    import re

    out = []
    pattern = re.compile(r'([A-Za-z]+):([0-9]+)')

    for g,d in lpos.groupby('comp'):
        if g != '':
            for i in range(len(g.split(' '))):
                tmp = d.copy()
                cn = pattern.search(g.split(' ')[i]).groups()
                tmp['element'] = cn[0]
                tmp['n'] = cn[1]
                out.append(tmp.copy())
    return pd.concat(out)

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
    
def unique(list1):
    unique_list = []       # initialize a null list
    for x in list1:    # traverse for all elements
        if x not in unique_list: # check if exists in unique_list or not
            unique_list.append(x)
    return unique_list

def data_for_cylinder_along_z(center_x,center_y,center_z,radius,height_z):
    z = np.linspace(center_z-height_z/2, center_z+height_z/2, 50)
    theta = np.linspace(0, 2*np.pi, 50)
    theta_grid, z_grid=np.meshgrid(theta, z)
    x_grid = radius*np.cos(theta_grid) + center_x
    y_grid = radius*np.sin(theta_grid) + center_y
    return x_grid,y_grid,z_grid

def gerade(k,x,d):
    return k*x+d

class APTyzer():                                                          # define class object 
    def __init__(self,tk):                                                     # initialize when starting tkinter
        self.directory=0
        self.plot_exist=0
        self.conc_check=0
        self.excess_check=0
        self.invert=0
        self.nexus=0
        self.color_map=['blue','skyblue','navy','red','salmon','orange','olive','cyan','crimson','green','yellow','purple','black','white','grey','brown','lime','gold','tan','plum','pink']
        self.data_color=['blue','red','green','yellow','purple','black','white','grey','brown'] # colors of cylinder to be seletected
        self.color_cp=['blue','red','green','yellow','purple','black','white','grey','brown']
        self.frame=Frame(tk)                                                   # setting up frame
        self.frame.pack(expand=True,fill=BOTH)                                 # placing frame
        self.canvas_all=Canvas(self.frame,width=1400,height=800,scrollregion=(0,0,1550,850))     #setting up canvas onto frame
        ################# scroll bars vertical and horizontal  ################
        self.hbar=Scrollbar(self.frame,orient=HORIZONTAL)                      # setting up horizontal scrollbar
        self.hbar.pack(side=TOP,fill=X)                                        # placing horizontal scrollbar on the top side
        self.hbar.config(command=self.canvas_all.xview)                        # configurating horizontal scrollbar
        self.vbar=Scrollbar(self.frame,orient=VERTICAL)                        # setting up vertical scrollbar
        self.vbar.pack(side=LEFT,fill=Y)                                       # placing vertical scrollbar on the left side
        self.vbar.config(command=self.canvas_all.yview)                        # configurating vertical scrollbar
        self.canvas_all.config(yscrollcommand=self.vbar.set,xscrollcommand=self.hbar.set)       #configurating canvas
        

        self.b_page1_loading = Button(self.canvas_all, text="1. Load", command=self.page_1,font=('helvetica',12,'bold')) # button for tip calculation
        self.canvas_all.create_window(75,20,window=self.b_page1_loading)
        
        self.b_page2_visualize = Button(self.canvas_all, text="2. Visualize", command=self.page_2,font=('helvetica',12,'bold')) # button for tip calculation
        self.canvas_all.create_window(180,20,window=self.b_page2_visualize)
        
        self.b_page3_cylinder = Button(self.canvas_all, text="3. Calculate", command=self.page_3,font=('helvetica',12,'bold')) # button for tip calculation
        self.canvas_all.create_window(300,20,window=self.b_page3_cylinder)

        ############## initiate values #######################################
        #### page 1 ####  
        self.succ=0
        n_large=12
        n_small=8
        self.utext0=Label(self.canvas_all,text='Welcome to the APTyzer!',font=('Times', n_large))    # title of segment
        self.utext1=Label(self.canvas_all,text='This open source tool enables you to look at/analyze APT tips',fg='grey',font=('Times', n_small))    # title of segment
        self.utext2=Label(self.canvas_all,text='All you need is a .pos (or .epos) file and a .rrng file',fg='grey',font=('Times', n_small))    # title of segment
        self.utext3=Label(self.canvas_all,text='Select the files you want to look at below and press "Load files"',fg='grey',font=('Times', n_small))    # title of segment
        self.utext4=Label(self.canvas_all,text='------------- Initiate files --------------',font=('Times', n_small))    # title of segment
        self.b_pos_file = Button(self.canvas_all,bg='green',fg='white', text="Select .pos file", command=self.search_pos)        # button for pos file initiation
        self.b_rrng_file = Button(self.canvas_all,bg='green',fg='white', text="Select .rrng file", command=self.search_rrng)             # button for rrng file initiation
        self.b_calc_tip = Button(self.canvas_all, text="Load files", command=self.calculate_tip,bg='blue',fg='white',font=('helvetica',14,'bold')) # button for tip calculation
        self.dtext0=Label(self.canvas_all,text='The loading of the files may take some time.',fg='grey',font=('Times', n_small))    # title of segment
        self.dtext1=Label(self.canvas_all,text='When "Sucessfully loaded files!" appears underneath the Load',fg='grey',font=('Times', n_small))    # title of segment
        self.dtext2=Label(self.canvas_all,text='button, then everthing is working as intendet',fg='grey',font=('Times', n_small))    # title of segment
        self.dtext3=Label(self.canvas_all,text='and a preview image of the tip should appear on the right',fg='grey',font=('Times', n_small))    # title of segment   
        self.dtext4=Label(self.canvas_all,text='You may now go to section 2.Visualize (top middle)',fg='grey',font=('Times', n_small))    # title of segment
        self.dtext5=Label(self.canvas_all,text='where you have more options for the visalization',fg='grey',font=('Times', n_small))    # title of segment
        self.dtext6=Label(self.canvas_all,text='as well as the ability to set control points and print them',fg='grey',font=('Times', n_small))    # title of segment
        self.dtext7=Label(self.canvas_all,text='After that you can go to section 3.Calculate (top right)',fg='grey',font=('Times', n_small))    # title of segment
        self.dtext8=Label(self.canvas_all,text='To calculate the concentration profile and excess profile',fg='grey',font=('Times', n_small))    # title of segment
        self.dtext9=Label(self.canvas_all,text='of a cylinder area(ROI) which you can select within the tip',fg='grey',font=('Times', n_small))    # title of segment
        self.label_help=Label(self.canvas_all,text='if you need help press this button:',fg='grey',font=('Times', n_small))    # title of segment
        self.b_help = Button(self.canvas_all, text="help",bg='orange',fg='white',command=self.help_web,font=('helvetica',12,'bold')) # button for tip calculation
        self.b_input_file = Button(self.canvas_all,bg='green',fg='white', text="Load existing APTyzer-input file", command=self.input_data)             # button for rrng file initiation
        self.b_nxs_file = Button(self.canvas_all,bg='green',fg='white', text="Select .nxs file", command=self.search_nxs) 
        
        
        #### page 2 ####
        self.cb_atom = Listbox(tk,selectmode = "multiple",height=6,exportselection='False',selectforeground='Black',selectbackground='Red')         
        self.diss=0
        self.diss2=0
        self.diss3=0
        self.slider_image_size_cv=DoubleVar()
        self.slider_image_size = Scale(self.canvas_all,from_=1,to=5,orient='horizontal',variable=self.slider_image_size_cv,showvalue=0)
        self.label_plot_tip=Label(self.canvas_all,text='------------- plot options ---------------')    # title of segment
        self.label_image_size=Label(self.canvas_all,text='Select image size')                     # label for resolution
        self.label_slider_res=Label(self.canvas_all,text='small')
        self.label_slider_res2=Label(self.canvas_all,text='large')     
        self.slider_image_size.set(3)
        self.b_plot1 = Button(self.canvas_all, text="plot tip", command=self.plot_tip,bg='blue',fg='white',font=('helvetica',12,'bold')) # button for plotting the tip
        self.var_axis=IntVar()
        self.check_axis = Checkbutton(self.canvas_all,text="Hide Axes", onvalue=1, offvalue=0,variable=self.var_axis,font=('helvetica', 10,'bold'))
        self.label_resolution=Label(self.canvas_all,text='Number of atoms shown:')                     # label for resolution
        self.label_select_atom=Label(self.canvas_all,text='Select atom type')
        self.slider_res_cv=DoubleVar()
        self.atom_number=StringVar(value='2000')
        self.entry_atom_number=Entry(self.canvas_all,textvariable=self.atom_number,width=6)
        self.label_slider_res4=Label(self.canvas_all,text='large')
        self.slider_res = Scale(self.canvas_all,from_=1,to=4,orient='horizontal',variable=self.slider_res_cv,showvalue=0)
        self.slider_res.set(1)
        self.label_select_atom_size=Label(self.canvas_all,text='Select atom size')
        self.slider_size_cv=DoubleVar()
        self.slider_size = Scale(self.canvas_all,from_=10,to=200,orient='horizontal',variable=self.slider_size_cv,showvalue=0)
        self.label_slider_res5=Label(self.canvas_all,text='small')
        self.label_slider_res6=Label(self.canvas_all,text='large')  
        self.slider_size.set(50)
        self.i_plot=IntVar()
        self.cb_inverse=Checkbutton(self.canvas_all, text='Flip Tip', onvalue=1, offvalue=0, variable=self.i_plot, font=('helvetica', 10,'bold'))   
        self.control=IntVar()
        self.label_control=Label(self.canvas_all,text='Create Control Points by "shift+left click":')
        self.text_box = Text(self.canvas_all,height=10, width=35)
        self.b_control_print = Button(self.canvas_all, text="export CP",bg='green', command=self.save,fg='white',font=('helvetica',10,'bold'))        # button for pos file initiation
        self.b_control_clear = Button(self.canvas_all, text="clear CP",bg='red', command=self.clear_text,fg='white',font=('helvetica',10,'bold'))        # button for pos file initiation
        self.var_points=IntVar()
        self.check_points = Checkbutton(self.canvas_all,text="plot with points", onvalue=1, offvalue=0,variable=self.var_points,font=('helvetica', 10,'bold'))
        self.var_points.set(1)
        self.b_adjust_cly = Button(self.canvas_all, text="adjust ROI according to points",bg='yellow',fg='black', command=self.adjust,font=('helvetica',10,'bold'))        # button for pos file initiation     
        self.cb_color_cp = Combobox(tk, values=self.color_cp, width=10) 
        self.cb_color_cp.current(0)
        self.var_plane=IntVar()
        self.label_color_cp=Label(self.canvas_all,text='color of control points')  
        self.cont_plane=Label(self.canvas_all,text='*When creating new controlpoints dont show fit plane!',font=('helvetica', 8,'bold'))  
        self.check_plane = Checkbutton(self.canvas_all,text="show fit plane and normal vector", onvalue=1, offvalue=0,variable=self.var_plane,font=('helvetica', 10,'bold'))
        self.keep=IntVar()
        self.cb_keep=Checkbutton(self.canvas_all, text='Keep Orientation', onvalue=0, offvalue=1, variable=self.keep, font=('helvetica', 10,'bold'))   
        self.i_rela=IntVar()
        self.cb_absolute=Checkbutton(self.canvas_all, text='absolute:', onvalue=1, offvalue=0, variable=self.i_rela, font=('helvetica', 10,'bold'))   
        self.cb_relative=Checkbutton(self.canvas_all, text='relative:', onvalue=0, offvalue=1, variable=self.i_rela, font=('helvetica', 10,'bold'))   
        self.label_proc=Label(self.canvas_all,text=' %') 
        self.atom_proc=StringVar(value='1')
        self.entry_atom_proc=Entry(self.canvas_all,textvariable=self.atom_proc,width=6)
        
        
        #### page 3 ####
        self.var_cyl=IntVar()
        self.label_conc=Label(self.canvas_all,text='-------- concentration calculation option  --------')#.place(x=20,y=400)    # title of segment
        self.var_cyl=IntVar()
        self.check_cylinder = Checkbutton(self.canvas_all,text="show ROI for concentration calculation", onvalue=1, offvalue=0,variable=self.var_cyl,font=('helvetica', 10,'bold'))#.place(x=20,y=220)
        
        self.b_calc_conc = Button(self.canvas_all, text="plot concentration", command=self.calc_con,bg='blue',fg='white',font=('helvetica',12,'bold'))   # button for calculating the concentration of cylinder
        self.b_calc_excess = Button(self.canvas_all, text="plot excess", command=self.calc_excess,bg='blue',fg='white',font=('helvetica',12,'bold'))    # button for calculating the concentration of cylinder
        self.b_plot_cyl = Button(self.canvas_all, text="plot atoms in ROI", command=self.calc_zoom,bg='blue',fg='white',font=('helvetica',12,'bold'))  # button for calculating the concentration of cylinder
        self.b_print_ex = Button(self.canvas_all, text="export excess", command=self.save_excess,bg='green',fg='white',font=('helvetica',12,'bold'))    # button for calculating the concentration of cylinder
        
        self.label_cly_pos=Label(self.canvas_all,text='cylinder center position')
        self.label_cly_x=Label(self.canvas_all,text='x')
        self.cly_x=StringVar(value='0')
        self.entry_cly_x=Entry(self.canvas_all,textvariable=self.cly_x,width=10)        
        self.label_cly_y=Label(self.canvas_all,text='y') 
        self.cly_y=StringVar(value='0')
        self.entry_cly_y=Entry(self.canvas_all,textvariable=self.cly_y,width=10)        
        self.label_cly_z=Label(self.canvas_all,text='z')
        self.cly_z=StringVar(value='0')
        self.entry_cly_z=Entry(self.canvas_all,textvariable=self.cly_z,width=10)
        self.label_cly_radius=Label(self.canvas_all,text='radius of cylinder')  
        self.radius=StringVar(value='10')
        self.entry_cly_radius=Entry(self.canvas_all,textvariable=self.radius)        
        self.label_cly_height=Label(self.canvas_all,text='height of cylinder') 
        self.height=StringVar(value='30')
        self.entry_cly_height=Entry(self.canvas_all,textvariable=self.height)        
        self.label_cly_beta=Label(self.canvas_all,text='tilt along x-axis')
        self.beta=StringVar(value='0')
        self.entry_cly_beta=Entry(self.canvas_all,textvariable=self.beta)
        self.label_cly_alpha=Label(self.canvas_all,text='tilt along y-axis')
        self.alpha=StringVar(value='0')
        self.entry_cly_alpha=Entry(self.canvas_all,textvariable=self.alpha)
        self.label_color_cyl=Label(self.canvas_all,text='color of the cylinder')   # label for cylinder color
        self.cb_color = Combobox(self.canvas_all, values=self.data_color)
        self.cb_color.current(0)            
        self.label_inter=Label(self.canvas_all,text='Select interval')      # label for resolution
        self.slider_inter_cv=DoubleVar()
        self.label_slider_inter=Label(self.canvas_all,text='small              large')    # title of segment
        self.slider_inter = Scale(self.canvas_all,from_=1,to=20,orient='horizontal',variable=self.slider_inter_cv,showvalue=0)#Combobox(self.canvas_all, values=self.data_res)                                                            # combobox for resolution
        self.slider_inter.set(15)
        self.b_plot2 = Button(self.canvas_all, text="export input data", command=self.save_input,bg='green',fg='white',font=('helvetica',12,'bold')) # button for plotting the tip
        self.b_print_conc = Button(self.canvas_all, text="export concentration", command=self.save_conc,bg='green',fg='white',font=('helvetica',12,'bold')) # button for plotting the tip
        self.var_norm=IntVar()
        self.check_norm = Checkbutton(self.canvas_all,text="normalize Excess", onvalue=1, offvalue=0,variable=self.var_norm)
        self.var_switch=IntVar()
        self.check_switch = Checkbutton(self.canvas_all,text="set cylinders         as bulk*", onvalue=1, offvalue=0,variable=self.var_switch)
        self.label_astrix=Label(self.canvas_all,text='* the concentration of each segment is set',fg='grey',font=('helvetica',9,'bold')) 
        self.label_astrix2=Label(self.canvas_all,text='in relation to the ends of the ROI',fg='grey',font=('helvetica',9,'bold'))  
        self.data_bulk=['start','end']
        self.data_bulk=['start','end']
        self.cb_bulk = Combobox(tk, values=self.data_bulk, width=5) 
        self.cb_bulk.current(0)
        self.text_box_ex = Text(self.canvas_all,height=6, width=25)
        self.label_exp=Label(self.canvas_all,text='For calcuating the excess select fit 4 points',font=('helvetica',9,'bold'))  
        self.label_exp2=Label(self.canvas_all,text='by "shift+left click" on the figure:',font=('helvetica',9,'bold'))  

        
        ############## setting up page 1 ##################
        self.a1=self.canvas_all.create_window(200,50,window=self.utext0)        
        self.a2=self.canvas_all.create_window(200,70,window=self.utext1)        
        self.a3=self.canvas_all.create_window(200,90,window=self.utext2)        
        self.a4=self.canvas_all.create_window(200,110,window=self.utext3)        
        self.a5=self.canvas_all.create_window(200,135,window=self.utext4)  
        self.a6=self.canvas_all.create_window(120,165,window=self.b_pos_file) 
        self.a7=self.canvas_all.create_window(280,165,window=self.b_rrng_file) 
        self.a8=self.canvas_all.create_window(200,230,window=self.b_calc_tip)
        self.a9=self.canvas_all.create_window(200,320,window=self.dtext0)        
        self.a10=self.canvas_all.create_window(200,340,window=self.dtext1)  
        self.a11=self.canvas_all.create_window(200,360,window=self.dtext2)   
        self.a12=self.canvas_all.create_window(200,380,window=self.dtext3)   
        self.a13=self.canvas_all.create_window(200,420,window=self.dtext4)        
        self.a14=self.canvas_all.create_window(200,440,window=self.dtext5)        
        self.a15=self.canvas_all.create_window(200,460,window=self.dtext6)
        self.a16=self.canvas_all.create_window(200,480,window=self.dtext7)
        self.a17=self.canvas_all.create_window(200,500,window=self.dtext8)
        self.a18=self.canvas_all.create_window(200,520,window=self.dtext9)
        self.a19=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a20=self.canvas_all.create_window(150,580,window=self.b_help)
        self.a21=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a22=self.canvas_all.create_window(200,290,window=self.b_input_file)
        self.a23=self.canvas_all.create_window(280,580,window=self.b_nxs_file)
        self.a24=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a25=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a26=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a27=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a28=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a29=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a30=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a31=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a32=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a33=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a34=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a35=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a36=self.canvas_all.create_window(200,540,window=self.label_help)
            
        ###################### setup figures  #################################                        
        self.fig=Figure(figsize=(17,13))                                       # defining figure size
        self.canvas=FigureCanvasTkAgg(self.fig,master=self.canvas_all)         # setting up canvas for figure
        self.toolbar=NavigationToolbar2Tk(self.canvas, self.canvas_all)        # setting up toolbar for figure
        self.canvas_all.create_window(400,10,window=self.canvas.get_tk_widget(),anchor=N+W,tags='canvas') #placing figure canvas on general canvas
        self.canvas_all.pack(expand=True,fill=BOTH)                            # placing general canvas at the end
        self.E=2                                                               # setting up variable to check if .pos/.epos file is selected
        self.R=2                                                               # setting up variable to check if .rrng file is selected       
              
    def page_1(self):
        self.canvas_all.delete(self.a1,self.a2,self.a3,self.a4,self.a5,self.a6,self.a7,self.a8,self.a9,self.a10,self.a11,self.a12,self.a13,self.a14,self.a15,self.a16,self.a17,self.a18,self.a19,self.a20,self.a21,self.a22,self.a23,self.a24,self.a25,self.a26,self.a27,self.a28,self.a29,self.a30,self.a31,self.a32,self.a33,self.a34,self.a35,self.a36)        
        if self.succ==1:self.a_succ=self.canvas_all.create_window(200,260,window=self.label_succ) #placing label on canvas
        if self.R==1:self.aRS=self.canvas_all.create_window(280,190,window=self.label_rrng_selected)
        if self.R==0:self.aR=self.canvas_all.create_window(280,190,window=self.label_rrng_selected)
        if self.E==0:self.aE=self.canvas_all.create_window(120,190,window=self.label_pos_selected)
        if self.E==1:self.aES=self.canvas_all.create_window(120,190,window=self.label_pos_selected)
        if self.nexus==1:self.aN=self.canvas_all.create_window(280,600,window=self.label_pos_nxs)   
        self.a1=self.canvas_all.create_window(200,50,window=self.utext0)        
        self.a2=self.canvas_all.create_window(200,70,window=self.utext1)        
        self.a3=self.canvas_all.create_window(200,90,window=self.utext2)        
        self.a4=self.canvas_all.create_window(200,110,window=self.utext3)        
        self.a5=self.canvas_all.create_window(200,135,window=self.utext4)  
        self.a6=self.canvas_all.create_window(120,165,window=self.b_pos_file) 
        self.a7=self.canvas_all.create_window(280,165,window=self.b_rrng_file) 
        self.a8=self.canvas_all.create_window(200,230,window=self.b_calc_tip)
        self.a9=self.canvas_all.create_window(200,320,window=self.dtext0)        
        self.a10=self.canvas_all.create_window(200,340,window=self.dtext1)  
        self.a11=self.canvas_all.create_window(200,360,window=self.dtext2)   
        self.a12=self.canvas_all.create_window(200,380,window=self.dtext3)   
        self.a13=self.canvas_all.create_window(200,420,window=self.dtext4)        
        self.a14=self.canvas_all.create_window(200,440,window=self.dtext5)        
        self.a15=self.canvas_all.create_window(200,460,window=self.dtext6)
        self.a16=self.canvas_all.create_window(200,480,window=self.dtext7)
        self.a17=self.canvas_all.create_window(200,500,window=self.dtext8)
        self.a18=self.canvas_all.create_window(200,520,window=self.dtext9)
        self.a19=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a20=self.canvas_all.create_window(150,580,window=self.b_help)
        self.a21=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a22=self.canvas_all.create_window(200,290,window=self.b_input_file)
        self.a23=self.canvas_all.create_window(280,580,window=self.b_nxs_file)
        self.a24=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a25=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a26=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a27=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a28=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a29=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a30=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a31=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a32=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a33=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a34=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a35=self.canvas_all.create_window(200,540,window=self.label_help)
        self.a36=self.canvas_all.create_window(200,540,window=self.label_help)
        
    def page_2(self):
        self.canvas_all.delete(self.a1,self.a2,self.a3,self.a4,self.a5,self.a6,self.a7,self.a8,self.a9,self.a10,self.a11,self.a12,self.a13,self.a14,self.a15,self.a16,self.a17,self.a18,self.a19,self.a20,self.a21,self.a22,self.a23,self.a24,self.a25,self.a26,self.a27,self.a28,self.a29,self.a30,self.a31,self.a32,self.a33,self.a34,self.a35,self.a36)
        if self.R==1:self.canvas_all.delete(self.aRS)
        if self.R==0:self.canvas_all.delete(self.aR)
        if self.E==0:self.canvas_all.delete(self.aE)
        if self.E==1:self.canvas_all.delete(self.aES)
        if self.succ==1: self.canvas_all.delete(self.a_succ)
        if self.nexus==1:self.canvas_all.delete(self.aN)
        ############## setting up buttons, Scales, comboboxes, etc. for plot options ###################    
        self.a1=self.canvas_all.create_window(200,50,window=self.label_plot_tip)        
        self.a2=self.canvas_all.create_window(100,80,window=self.label_image_size)        
        self.a3=self.canvas_all.create_window(30,105,window=self.label_slider_res)        
        self.a4=self.canvas_all.create_window(170,105,window=self.label_slider_res2)        
        self.a5=self.canvas_all.create_window(100,105,window=self.slider_image_size)        
        self.a6=self.canvas_all.create_window(250,300,window=self.b_plot1)        
        self.a7=self.canvas_all.create_window(280,200,window=self.check_axis)        
        self.a8=self.canvas_all.create_window(100,145,window=self.label_resolution)        
        self.a9=self.canvas_all.create_window(280,70,window=self.label_select_atom)    
        self.a10=self.canvas_all.create_window(175,170,window=self.label_proc)
        self.a11=self.canvas_all.create_window(150,200,window=self.entry_atom_number)   
        self.a12=self.canvas_all.create_window(150,170,window=self.entry_atom_proc)        
        self.a13=self.canvas_all.create_window(100,230,window=self.label_select_atom_size)        
        self.a14=self.canvas_all.create_window(30,250,window=self.label_slider_res5)        
        self.a15=self.canvas_all.create_window(170,250,window=self.label_slider_res6)
        self.a16=self.canvas_all.create_window(100,250,window=self.slider_size)        
        self.a17=self.canvas_all.create_window(272,230,window=self.cb_inverse)        
        self.a18=self.canvas_all.create_window(280,130,window=self.cb_atom)             # putting combobox with element selection on canvas
        self.a19=self.canvas_all.create_window(200,350,window=self.label_control)        
        self.a20=self.canvas_all.create_window(200,450,window=self.text_box)        
        self.a21=self.canvas_all.create_window(80,560,window=self.b_control_print)        
        self.a22=self.canvas_all.create_window(320,560,window=self.b_control_clear)        
        self.a23=self.canvas_all.create_window(200,560,window=self.check_points)  
        self.a24=self.canvas_all.create_window(200,600,window=self.b_adjust_cly)
        self.a25=self.canvas_all.create_window(100,310,window=self.cb_color_cp)
        self.a26=self.canvas_all.create_window(100,290,window=self.label_color_cp)
        self.a27=self.canvas_all.create_window(200,630,window=self.check_plane) 
        self.a28=self.canvas_all.create_window(300,260,window=self.cb_keep)
        self.a29=self.canvas_all.create_window(200,650,window=self.cont_plane)
        self.a30=self.canvas_all.create_window(80,200,window=self.cb_absolute)
        self.a31=self.canvas_all.create_window(80,170,window=self.cb_relative)
        self.a32=self.canvas_all.create_window(170,250,window=self.label_slider_res6)
        self.a33=self.canvas_all.create_window(170,250,window=self.label_slider_res6)
        self.a34=self.canvas_all.create_window(170,250,window=self.label_slider_res6)
        self.a35=self.canvas_all.create_window(170,250,window=self.label_slider_res6)
        self.a36=self.canvas_all.create_window(170,250,window=self.label_slider_res6)

    def page_3(self):
        self.canvas_all.delete(self.a1,self.a2,self.a3,self.a4,self.a5,self.a6,self.a7,self.a8,self.a9,self.a10,self.a11,self.a12,self.a13,self.a14,self.a15,self.a16,self.a17,self.a18,self.a19,self.a20,self.a21,self.a22,self.a23,self.a24,self.a25,self.a26,self.a27,self.a28,self.a29,self.a30,self.a31,self.a32,self.a33,self.a34,self.a35,self.a36)        
        if self.R==1:self.canvas_all.delete(self.aRS)
        if self.R==0:self.canvas_all.delete(self.aR)
        if self.E==0:self.canvas_all.delete(self.aE)
        if self.E==1:self.canvas_all.delete(self.aES)
        if self.succ==1: self.canvas_all.delete(self.a_succ)
        if self.nexus==1:self.canvas_all.delete(self.aN)
        ###########  setting up buttons, Scales, comboboxes etc. for cylinder selection and concentration calculations  ###################
        #self.label_exp=Label(self.canvas_all,text='For calcuating the excess select 4 points') 
        self.entry_cly_x=Entry(self.canvas_all,textvariable=self.cly_x,width=10) 
        self.entry_cly_y=Entry(self.canvas_all,textvariable=self.cly_y,width=10)
        self.entry_cly_z=Entry(self.canvas_all,textvariable=self.cly_z,width=10)
        self.entry_cly_alpha=Entry(self.canvas_all,textvariable=self.alpha)
        self.entry_cly_beta=Entry(self.canvas_all,textvariable=self.beta)
        
        self.a1=self.canvas_all.create_window(200,50,window=self.label_conc)        
        self.a2=self.canvas_all.create_window(200,90,window=self.check_cylinder)        
        self.a3=self.canvas_all.create_window(100,450,window=self.b_calc_conc)        
        self.a4=self.canvas_all.create_window(100,500,window=self.b_calc_excess)        
        self.a5=self.canvas_all.create_window(100,400,window=self.b_plot_cyl)    
        self.a6=self.canvas_all.create_window(300,500,window=self.b_print_ex) 
        self.a7=self.canvas_all.create_window(200,120,window=self.label_cly_pos)    
        self.a8=self.canvas_all.create_window(100,140,window=self.label_cly_x)        
        self.a9=self.canvas_all.create_window(100,170,window=self.entry_cly_x)      
        self.a10=self.canvas_all.create_window(200,140,window=self.label_cly_y)        
        self.a11=self.canvas_all.create_window(200,170,window=self.entry_cly_y)        
        self.a12=self.canvas_all.create_window(300,140,window=self.label_cly_z)        
        self.a13=self.canvas_all.create_window(300,170,window=self.entry_cly_z)
        self.a14=self.canvas_all.create_window(100,200,window=self.label_cly_radius)
        self.a15=self.canvas_all.create_window(100,225,window=self.entry_cly_radius)                       
        self.a16=self.canvas_all.create_window(300,200,window=self.label_cly_height)        
        self.a17=self.canvas_all.create_window(300,225,window=self.entry_cly_height)         
        self.a18=self.canvas_all.create_window(100,255,window=self.label_cly_beta)        
        self.a19=self.canvas_all.create_window(100,280,window=self.entry_cly_beta)        
        self.a20=self.canvas_all.create_window(300,255,window=self.label_cly_alpha)      
        self.a21=self.canvas_all.create_window(300,280,window=self.entry_cly_alpha)             
        self.a22=self.canvas_all.create_window(100,330,window=self.label_color_cyl)        
        self.a23=self.canvas_all.create_window(100,355,window=self.cb_color)        
        self.a24=self.canvas_all.create_window(300,310,window=self.label_inter)        
        self.a25=self.canvas_all.create_window(300,355,window=self.label_slider_inter)        
        self.a26=self.canvas_all.create_window(300,355,window=self.slider_inter)        
        self.a27=self.canvas_all.create_window(300,400,window=self.b_plot2)        
        self.a28=self.canvas_all.create_window(300,450,window=self.b_print_conc)
        self.a29=self.canvas_all.create_window(80,550,window=self.check_norm)
        self.a30=self.canvas_all.create_window(260,550,window=self.check_switch)
        self.a31=self.canvas_all.create_window(220,570,window=self.label_astrix)
        self.a32=self.canvas_all.create_window(220,590,window=self.label_astrix2)
        self.a33=self.canvas_all.create_window(280,550,window=self.cb_bulk)
        self.a34=self.canvas_all.create_window(200,610,window=self.label_exp)
        self.a35=self.canvas_all.create_window(200,630,window=self.label_exp2)
        self.a36=self.canvas_all.create_window(200,675,window=self.text_box_ex)
        
    def help_web(self):
        url='https://github.com/areichm/APTyzer'
        new=1
        webbrowser.open(url,new=new)
        
    def clear_text(self):    
        self.text_box.delete('1.0', END)
                
    def save(self):
        points=self.text_box.get(1.0,END)
        points=points.split('\n')
        points=[x for x in points if x!='']
        xyz=[]
        for i in range(0,len(points)):
            points_inner=points[i].split(',')
            for j in points_inner:
                m=j.split('=')
                xyz.append(m[1])
        data=np.array(xyz).reshape(int(len(xyz)/3),3)
        label=('x','y','z')      
        df=pd.DataFrame(data,columns=label) 
        path=self.directory+"/controlpoints.h5"
        df.to_hdf(path,'data')
        messagebox.showinfo(title='APTyzer',message='Control points saved to hdf5 file: "controlpoints.h5" in the same directory as the .pos/.epos file.')

    def save_conc(self):
        if self.conc_check==0:messagebox.showinfo(title='APTyzer',message='please plot concentration first')
        else: 
            path=self.directory+'/APTyzer_concentration.csv'
            self.df_con.to_csv(path)
            messagebox.showinfo(title='APTyzer',message='Concentration profile saved to csv file: "APTyzer_concentration.csv" in the same directory as the .pos/.epos file.')
    
    def save_input(self):
        file=filedialog.asksaveasfilename(filetypes=[("txt file", ".txt")],defaultextension=".txt",initialdir=r'/config/uploads')
        if file:  # user selected file
            f=open(file,'w')
            f.write('position and size of ROI:\n')
            f.write('x position:\n')
            f.write(self.cly_x.get())
            f.write('\n')
            f.write('y position:\n')
            f.write(self.cly_y.get())
            f.write('\n')
            f.write('z position:\n')
            f.write(self.cly_z.get())
            f.write('\n')
            f.write('height:\n')
            f.write(self.height.get())
            f.write('\n')
            f.write('radius:\n')
            f.write(self.radius.get())
            f.write('\n')
            f.write('alpha angle:\n')
            f.write(self.alpha.get())
            f.write('\n')
            f.write('beta angle:\n')
            f.write(self.beta.get())
            f.write('\n')
            f.write('control points:\n')
            f.write(self.text_box.get(1.0,END))
            f.write('excess fit points:\n')
            f.write(self.text_box_ex.get(1.0,END))
            f.close()
            messagebox.showinfo(title='APTyzer',message='All input parameters have been saved to .txt file')
        else: # user cancel the file browser window
            messagebox.showinfo(title='APTyzer',message='No file chosen')
            
    def input_data(self):
        if self.directory==0:
            self.filename3=filedialog.askopenfilename(initialdir=r'/config/uploads')                             # getting file from search
            #self.filename3=filedialog.askopenfilename(filetypes=[("Text file", ".txt"),("all files","*.*")])
        else:
            self.filename3=filedialog.askopenfilename(initialdir=self.directory,filetypes=[("Text file", ".txt"),("all files","*.*")])
        with open(self.filename3, 'r') as f:
            inp = f.readlines()
        self.cly_x.set(float(inp[2]))
        self.cly_y.set(float(inp[4]))
        self.cly_z.set(float(inp[6]))
        self.height.set(float(inp[8]))
        self.radius.set(float(inp[10]))
        self.alpha.set(float(inp[12]))
        self.beta.set(float(inp[14]))
        index_con=inp.index('control points:\n')
        index_exc=inp.index('excess fit points:\n')
        for i in range(index_con+1,index_exc):
            self.text_box.insert(END,inp[i])
        for i in range(index_exc+1,len(inp)):
            self.text_box_ex.insert(END,inp[i])
        
        
    def save_excess(self):
        if self.excess_check==0:messagebox.showinfo(title='APTyzer',message='please plot excess first')
        else: 
            path2=self.directory+'/APTyzer_excess.csv'
            self.df_ex.to_csv(path2)
            messagebox.showinfo(title='APTyzer',message='Excess profile saved to csv file: "APTyzer_concentration.csv" in the same directory as the .pos/.epos file.')
                      
    def adjust(self):                   # adjust cylinder according to controlpoints
        self.var_cyl.set(1)
        XX=[]                           # setting up arrays
        YY=[]
        ZZ=[]
        points=self.text_box.get(1.0,END)   # get text from text box
        points=points.split('\n')           # split the text into each row
        points=[x for x in points if x!=''] # remove any rows that dont contain information
        for i in range(0,len(points)):      # loop over all rows
                points_inner=points[i].split(',')   # split each row into parts of each coordinates
                xyz2=[]
                for j in points_inner:               # loop over all values inside row
                    m=j.split('=')                   # isolate actual number
                    xyz2.append(m[1])                # save number
                XX.append(float(xyz2[0]))            # save x,y and z coordinate
                YY.append(float(xyz2[1]))
                ZZ.append(float(xyz2[2]))
        
        if len(XX)>=3:                               # if 3 or more control points are chosen
                center_x=round(sum(XX)/len(XX),3)             # calculate center in x, y and z
                center_y=round(sum(YY)/len(YY),3)
                center_z=round(sum(ZZ)/len(ZZ),3)
                self.cly_x=StringVar(value=center_x)                    #save the center as coordinates for cylinder
                self.cly_y=StringVar(value=center_y)
                self.cly_z=StringVar(value=center_z)
                tmp_C=[]
                for i in range(len(XX)):
                    tmp_C.append([XX[i],YY[i],ZZ[i]])                              #write x,y and z into one matrix and transpose it below
                C=np.array(tmp_C).T
                svd=np.linalg.svd(C - np.mean(C, axis=1, keepdims=True))           # using singular value decomposition to find fit
                left=svd[0]                                                        # upper line has to substract out the centroid first
                fit=left[:,-1]
                normal=np.array([fit[0],fit[1],fit[2]])
                norm=np.linalg.norm(normal)
                n_n=normal/norm                                                    # select fitted values
                aa=np.arcsin(-n_n[1])
                bb=math.atan2(n_n[0],n_n[2])     # this is done by applying rotation matrix in reverse order onto normal verctor
                                                 # and then solving the equation for alpha and beta
                #al=round(math.degrees(aa),4)    # that rotate the normal vector into x=0 and y=0
                #if normal[2]<0:al=round(math.degrees(-aa),4)
                al=round(math.degrees(aa),4)
                be=round(math.degrees(bb),4)
                self.alpha=StringVar(value=al)                                     # then transforming into degree and  
                self.beta=StringVar(value=be)                                      # saving alpha and beta values into entry field
        else: messagebox.showinfo(title='APTyzer',message='Choose at least 3 control points')  # if less than 3 points are selected show message
                
    
    def search_nxs(self):                                                      # function for searching for pos/epos file
        if self.nexus==1:self.canvas_all.delete(self.aN)
        self.nexus==0
        if self.directory==0:
            self.filename3=filedialog.askopenfilename(initialdir=r'/config/uploads',filetypes=[("Nexus file", ".nxs .NXS"),("all files","*.*")])                             # getting file from search
            #self.filename=filedialog.askopenfilename(filetypes=[("Position file", ".pos .POS .epos .EPOS"),("all files","*.*")])
            self.directory = os.path.split(self.filename3)[0]
        else:
            self.filename3=filedialog.askopenfilename(initialdir=self.directory,filetypes=[("Nexus file", ".nxs .NXS"),("all files","*.*")])
            self.directory = os.path.split(self.filename3)[0]
        N='.NXS' in self.filename3                                              # checking if .POS is in the filename
        n='.nxs' in self.filename3                                              # checking if .pos is in the filename
        if N or n is True:                                                     # if .POS or .pos was selected
            if self.E==0:self.canvas_all.delete(self.aE)
            if self.E==1:self.canvas_all.delete(self.aES)
            if self.R==1:self.canvas_all.delete(self.aRS)
            if self.R==0:self.canvas_all.delete(self.aR)
            self.label_pos_nxs=Label(text='nexus file selected')            # create label for pos
            self.nexus=1
            self.aN=self.canvas_all.create_window(280,600,window=self.label_pos_nxs) #placing label on canvas
            self.N=1                                                           # set variable for later recognition
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
            self.E=2 
            self.R=2 
        else:                                                                  # if any other filetype was selected
            messagebox.showinfo(title='APTyzer',message='select .nxs file. Warning: Notation has to be either .nxs or .NXS')
            self.filename3=[]                                                   # leave filename empty
        
#     
    def search_pos(self):                                                      # function for searching for pos/epos file
        if self.E==0:self.canvas_all.delete(self.aE)
        if self.E==1:self.canvas_all.delete(self.aES)
        if self.directory==0:
            self.filename=filedialog.askopenfilename(initialdir=r'/config/uploads',filetypes=[("Position file", ".pos .POS .epos .EPOS"),("all files","*.*")])                             # getting file from search
            #self.filename=filedialog.askopenfilename(filetypes=[("Position file", ".pos .POS .epos .EPOS"),("all files","*.*")])
            self.directory = os.path.split(self.filename)[0]
        else:
            self.filename=filedialog.askopenfilename(initialdir=self.directory,filetypes=[("Position file", ".pos .POS .epos .EPOS"),("all files","*.*")])
            self.directory = os.path.split(self.filename)[0]
        S='.POS' in self.filename                                              # checking if .POS is in the filename
        ES='.EPOS' in self.filename                                            # checking if .EPOS is in the filename
        s='.pos' in self.filename                                              # checking if .pos is in the filename
        es='.epos' in self.filename                                            # checking if .epos is in the filename
        if S or s is True:                                                     # if .POS or .pos was selected
            if self.nexus==1:self.canvas_all.delete(self.aN)
            self.nexus=0
            self.label_pos_selected=Label(text='pos file selected')            # create label for pos
            self.aE=self.canvas_all.create_window(120,190,window=self.label_pos_selected) #placing label on canvas
            self.E=0                                                           # set variable for later recognition
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
        elif ES or es is True:                                                 # if .EPOS or .epos was selected
            if self.nexus==1:self.canvas_all.delete(self.aN)
            self.nexus=0
            self.label_pos_selected=Label(text='epos file selected')           # creating label for epos
            self.aES=self.canvas_all.create_window(120,190,window=self.label_pos_selected) #put label on canvas
            self.E=1                                                           # set variable for later recognition
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
        else:                                                                  # if any other filetype was selected
            messagebox.showinfo(title='APTyzer',message='select .pos or .epos file. Warning: Notation has to be either .pos /.epos or .POS / .EPOS')
            self.filename=[]                                                   # leave filename empty
            self.E=2                                                           # set variable back to 2
            
    def search_rrng(self):                                                     # function for searching for rrng file
        if self.R==1:self.canvas_all.delete(self.aRS)
        if self.R==0:self.canvas_all.delete(self.aR)
        if self.directory==0:
            self.filename2=filedialog.askopenfilename(initialdir=r'/config/uploads')                             # getting file from search
            #self.filename2=filedialog.askopenfilename(filetypes=[("Range file", ".rrng .RRNG"),("all files","*.*")])
        else:
            self.filename2=filedialog.askopenfilename(initialdir=self.directory,filetypes=[("Range file", ".rrng .RRNG"),("all files","*.*")])
        S='.RRNG' in self.filename2                                            # checking if .RRNG is in the filename
        s='.rrng' in self.filename2                                            # checking if .rrng is in the filename
        SR='.RNG' in self.filename2
        sr='.rng' in self.filename2
        if S or s is True:                                                     # if .RRNG or .rrng was selected
            if self.nexus==1:self.canvas_all.delete(self.aN)
            self.nexus=0
            self.label_rrng_selected=Label(text='.rrng file selected')         # create label for rrng
            self.aR=self.canvas_all.create_window(280,190,window=self.label_rrng_selected) #placing label on canvas
            self.R=0                                                           # setting variable for later recognition
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
        elif SR or sr is True:
            if self.nexus==1:self.canvas_all.delete(self.aN)
            self.nexus=0
            self.label_rrng_selected=Label(text='.rng file selected')         # create label for rrng
            self.aRS=self.canvas_all.create_window(280,190,window=self.label_rrng_selected) #placing label on canvas
            self.R=1 
            if self.succ==1: self.canvas_all.delete(self.a_succ) 
            self.succ=0
        else:                                                                  # if no correct file was selected
            messagebox.showinfo(title='APTyzer',message='select .rrng file. Warning: Notation has to be either .rrng or .RRNG')
            self.filename2=[]                                                   # leaving filename empty
            self.R=2                                                           # set variable back to 2
        
    def calculate_tip(self):                                                   # function for reading and converting the pos and rrng file and creating the tip
        if self.nexus==0:
            if self.E==2:messagebox.showinfo(title='APTyzer',message='no file .pos/.epos file selected')   # checking if no  pos/epos file is selected (E=2)
            elif self.R==2:messagebox.showinfo(title='APTyzer',message='no file .rrng file selected')    # checking if no  rrng file is selected (R=2)
            else:                                                                    # if pos/epos and rrng file is selected
                if self.succ==1: self.canvas_all.delete(self.a_succ)
                with open(self.filename, 'rb') as f:                                   # opening pos/epos data from file1
                    data = f.read()                                                    # reading pos/epos data from file1
                data_rrng=self.filename2                                               # reading rrng data from file2
                if self.E==0:pos=read_pos(data)                                    # converting pos data if pos file was selected (E==0)
                elif self.E==1:pos=read_epos(data)                                 # converting epos data if epos file was selected (E==1)
                if self.R==0:ions,rrngs=read_rrng(data_rrng)                           # converting rrng data
                elif self.R==1:ions,rrngs=read_rng(data_rrng)
                self.tip=label_ions(pos, rrngs)                                    # label ions using apt_importers functions
                self.ele=unique(list(self.tip['comp']))                            # determining how many unique elements in tip using apt_importers functions
                self.string=['All']                                                         # setting up strings for atom selection combobox
                for i in range (0,len(self.ele)):                                      # loop over all unique elements
                    if self.ele[i]=='':
                        unn='Unnamed'+str(i)
                        self.string.append(unn)
                    else:
                        self.string.append(self.ele[i])                                         # creating strings with all unique elements
                self.data_atom=(self.string)                                                # data for combobox containing all unique elements + 'all' at start
                self.cb_atom.delete(0,END)
                for item in range(len(self.data_atom)): 
                    self.cb_atom.insert(END, self.data_atom[item]) 
                    self.cb_atom.itemconfig(item, bg="#bdc1d6")
                self.atom=1
                
                self.cb_atom.select_set(0)                                                # setting initial value of combobox to 'all'  
                self.fig.clear()                                                   # if plot single was selected clear all previous plots
                self.canvas_all.delete('message') 
                self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[2,1],wspace=0.5,hspace=0.5,height_ratios=[2,1]) #adjust the size of the figure compared to (111)
                self.ax=self.fig.add_subplot(self.spec[0],projection='3d')
                N=5000  
                u_ele=self.ele
                x=np.array(self.tip['x'])[::N]                                         # select x cooridant (only take every Nth atom to plot later)
                y=np.array(self.tip['y'])[::N]                                         # select y cooridant from tip data
                z=np.array(self.tip['z'])[::N]                                         # select z cooridant from tip data
                c=np.array(self.tip['colour'])[::N]                                    # select the colour column in the tip data
                label=np.array(self.tip['comp'])[::N]                                  # select the composition column in the tip data
                nn=int((len(u_ele)/len(self.color_map)))
                self.color_mapn=self.color_map.copy()
                for j in range(0,nn):
                    self.color_mapn.extend(self.color_map)
                for i in range(0,len(u_ele)):                                          # loop that goes over every different elements
                    e=label==u_ele[i]                                                 # seperation of the elements
                    x_new=x[e]                                                        # select the x values of each element indiviually
                    y_new=y[e]
                    z_new=z[e]
                    c_new=c[e]
                    self.ax.scatter3D(x_new,y_new,z_new,c=c_new,label=u_ele[i],s=1)  # scatter plot each element seperatly and assign label
                    x_N=len(x) 
                    self.ax.set_title('Preview (number of atoms={:.0f},  1/{:.0f} of all atoms)'.format(x_N,N))                                                   # assigne x_N value to show the number of plotted atoms in title
                self.canvas.draw_idle()
                set_axes_equal(self.ax)                                            # function for setting the axes equal for 3d plot
                self.ax.legend(loc='center left', bbox_to_anchor=(1.07, 0.5))#, fontsize=7)                                                        # show legends of each element   
                self.ax.set_xlabel('X-axis',fontweight='bold')                     # label x axis
                self.ax.set_ylabel('Y-axis',fontweight='bold')                # label y axis
                self.ax.set_zlabel('Z-axis',fontweight='bold')                     # label z axis
                self.label_succ=Label(text='Sucessfully loaded files!')         # create label for rrng
                self.a_succ=self.canvas_all.create_window(200,260,window=self.label_succ) #placing label on canvas
                self.succ=1

                label2=np.array(self.tip['comp'])
                message='Atoms in specimen:'
                message=message+'\n'
                for i in range(0,len(np.array(self.string))):
                    if self.string[i]=='All':
                        numm=len(self.tip['x'])
                    else:
                        e=label2==u_ele[i-1]
                        numm=len(self.tip['x'][e])
                    message=message+self.string[i]+'   '+str(numm)
                    message=message+'\n'
                self.plot_exist=0
                self.message=Text(height = 15,width = 26,relief=FLAT)
                self.message.insert(END, message)
                self.message.configure(state='disabled')
                self.canvas_all.create_window(510,140,window=self.message,tags='message')
        else:
            h5r = h5py.File(self.filename3, 'r')
            xyz = h5r['/entry1/atom_probe/reconstruction/reconstructed_positions'][:, :]
            mass = h5r['/entry1/atom_probe/mass_to_charge_conversion/mass_to_charge'][:, :]
            pos = pd.DataFrame({'x': xyz[:, 0],
                    'y': xyz[:, 1],
                    'z': xyz[:, 2],
                    'Da': mass[:, 0]})
            ions = []
            rrngs = []
            n_ions = h5r['/entry1/atom_probe/ranging/number_of_ion_types'][()]
            k = 1
            for i in np.arange(1, n_ions + 1):
                grpnm = '/entry1/atom_probe/ranging/peak_identification/ion' + str(i)
                nuclids = h5r[grpnm + '/nuclid_list'][1, :]
                values, counts = np.unique(nuclids[nuclids != 0], return_counts=True)
                vol = 0.  # not stored in a ranging information within NXapm
                comp_str = ''
                for j in np.arange(0, len(values)):
                    comp_str += ' ' + chemical_symbols[values[j]] + ':' + str(counts[j])
                color = 'FFFFFF'  # not stored in a ranging information within NXapm
                mq = h5r[grpnm + '/mass_to_charge_range'][:, :]
                for j in np.arange(0, np.shape(mq)[0]):
                    rrngs.append([k, mq[j, 0], mq[j, 1], vol, comp_str, color])
                    k += 1
            h5r.close()
            rrngs = pd.DataFrame(rrngs, columns=['number','lower','upper','vol','comp','colour'])
            rrngs.set_index('number',inplace=True)
            rrngs[['lower','upper','vol']] = rrngs[['lower','upper','vol']].astype(float)
            rrngs[['comp','colour']] = rrngs[['comp','colour']].astype(str)
            
            self.tip=label_ions(pos, rrngs)                                    # label ions using apt_importers functions
            #print(self.tip)
            self.ele=unique(list(self.tip['comp']))                            # determining how many unique elements in tip using apt_importers functions
            self.string=['All']
            for i in range (0,len(self.ele)):                                      # loop over all unique elements
                if self.ele[i]=='':
                    unn='Unnamed'+str(i)
                    self.string.append(unn)
                else:
                    self.string.append(self.ele[i])                                         # creating strings with all unique elements
            self.data_atom=(self.string)                                                # data for combobox containing all unique elements + 'all' at start
            self.cb_atom.delete(0,END)
            for item in range(len(self.data_atom)): 
                self.cb_atom.insert(END, self.data_atom[item]) 
                self.cb_atom.itemconfig(item, bg="#bdc1d6")
            self.atom=1

            self.cb_atom.select_set(0)                                                # setting initial value of combobox to 'all'  
            self.fig.clear()                                                   # if plot single was selected clear all previous plots
            self.canvas_all.delete('message') 
            self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[2,1],wspace=0.5,hspace=0.5,height_ratios=[2,1]) #adjust the size of the figure compared to (111)
            self.ax=self.fig.add_subplot(self.spec[0],projection='3d')
            N=5000
            u_ele=self.ele
            nn=int((len(u_ele)/len(self.color_map)))
            self.color_mapn=self.color_map.copy()
            for j in range(0,nn):
                self.color_mapn.extend(self.color_map)
            x=np.array(self.tip['x'])[::N]                                         # select x cooridant (only take every Nth atom to plot later)
            y=np.array(self.tip['y'])[::N]                                         # select y cooridant from tip data
            z=np.array(self.tip['z'])[::N]                                         # select z cooridant from tip data
            c=np.array(self.tip['colour'])[::N]                                    # select the colour column in the tip data
            label=np.array(self.tip['comp'])[::N]                                  # select the composition column in the tip data 
            for i in range(0,len(u_ele)):                                          # loop that goes over every different elements
                e=label==u_ele[i]                                                 # seperation of the elements
                x_new=x[e]                                                        # select the x values of each element indiviually
                y_new=y[e]
                z_new=z[e]
                c_new=c[e] 
                c_new[c_new=='#FFFFFF']=self.color_mapn[i]
                col=self.color_mapn[i-j*i]
                self.ax.scatter3D(x_new,y_new,z_new,c=c_new,label=u_ele[i]+' |  color: '+col,s=1)  # scatter plot each element seperatly and assign label
                x_N=len(x) 
                self.ax.set_title('Preview (number of atoms={:.0f},  1/{:.0f} of all atoms)'.format(x_N,N))                                                   # assigne x_N value to show the number of plotted atoms in title
            self.canvas.draw_idle()
            set_axes_equal(self.ax)                                            # function for setting the axes equal for 3d plot
            self.ax.legend(loc='center left', bbox_to_anchor=(1.07, 0.5))#, fontsize=7)                                                        # show legends of each element   
            self.ax.set_xlabel('X-axis',fontweight='bold')                     # label x axis
            self.ax.set_ylabel('Y-axis',fontweight='bold')                # label y axis
            self.ax.set_zlabel('Z-axis',fontweight='bold')                     # label z axis
            self.label_succ=Label(text='Sucessfully loaded files!')         # create label for rrng
            self.a_succ=self.canvas_all.create_window(200,260,window=self.label_succ) #placing label on canvas
            self.succ=1

            label2=np.array(self.tip['comp'])
            message='Atoms in specimen:'
            message=message+'\n'
            for i in range(0,len(np.array(self.string))):
                if self.string[i]=='All':
                    numm=len(self.tip['x'])
                else:
                    e=label2==u_ele[i-1]
                    numm=len(self.tip['x'][e])
                message=message+self.string[i]+'   '+str(numm)
                message=message+'\n'
            self.plot_exist=0
            self.message=Text(height = 15,width = 26,relief=FLAT)
            self.message.insert(END, message)
            self.message.configure(state='disabled')
            self.canvas_all.create_window(510,140,window=self.message,tags='message')
            
    def clear(self):                                                           # function for clearing canvas
        self.fig.clear()                                                       # clearing canvas
        self.canvas.draw_idle()                                                # drawing 
        self.canvas_all.delete('message')                                      # deleting message about concentration data
        
    def plot_tip(self):                               # function about plotting tip
        # read in old plot, clear it and then set values of new plot to old one
        if self.plot_exist==1:  # if plot already exists get the parameters of old plot
            self.fig.canvas.mpl_disconnect(self.cid)            
            if self.ax.get_xlim()!=(0,1) and self.keep.get()==0:
                azim=self.ax.azim                                                  # get azimutal angle of old plot
                elev=self.ax.elev
                xlim=self.ax.get_xlim()                                           # get x,y and z limit of old plot
                ylim=self.ax.get_ylim()
                zlim=self.ax.get_zlim()
        if self.diss2==1:
            self.fig.canvas.mpl_disconnect(self.cid2)
            self.diss2=0
        if self.i_plot.get()==0:self.invert=0                                 # if inversion checkbox is unmarked set value=0
        else: self.invert=1                                                   # if inversion checkbox is marked set value=1
        self.fig.clear()                                                   # if plot single was selected clear all previous plots
        image_size=self.slider_image_size.get()
        image_size=float(image_size)**2
        self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[image_size,1],wspace=0.1,hspace=0.1,height_ratios=[image_size,1]) #adjust the size of the figure compared to (111)
        self.ax=self.fig.add_subplot(self.spec[0],projection='3d')         # make new figure with size depending on the image size slider
        if self.plot_exist==1 and self.keep.get()==0:   # if plot already exist set old parameters of plot as new ones
            if xlim!=(0,1):
                self.ax.view_init(elev=elev,azim=azim)                         # set azimuthal angle and elevation of old plot as new one
                self.ax.set_xlim3d(xlim[0],xlim[1])                            # set x,y and z limit of old plot as new one
                self.ax.set_ylim3d(ylim[0],ylim[1])
            if self.i_plot.get()==0 and self.invert==0:self.ax.set_zlim3d(zlim[0],zlim[1])  # depending on if the plot should be inverted or not
            if self.i_plot.get()==1 and self.invert==1:self.ax.set_zlim3d(zlim[0],zlim[1])  # reset the z limit of new plot to center it again
                                                              # set plot exist as one in order to mark that plot exists      
        color_zyl=str(self.cb_color.get())                                    # get color of cylinder from color entry box
        color_cp=str(self.cb_color_cp.get())
        self.canvas_all.delete('message')                                      # clearing message about concentration data

        Z= self.cb_atom.curselection()                                         # getting variable from listbox of atom selection    
        if Z==(): 
            self.cb_atom.select_set(0) 
            Z= self.cb_atom.curselection() 
        index=np.array(Z)
        atoms=np.array(self.string)
        #M = self.slider_res.get()                                              # getting variable from combobox of resolution selection

        SS=self.slider_size.get()                                              # getting variable from combobox of atom size selection
        S=float(SS)/100                                                        # transforming entry into float and deviding by 10 so values 0.1 to 1 are also included
        image_size=self.slider_image_size.get()
        image_size=float(image_size)**2
        u_ele=self.ele
        
        x=np.array(self.tip['x'])
        y=np.array(self.tip['y'])
        z=np.array(self.tip['z'])
        c=np.array(self.tip['colour'])
        label=np.array(self.tip['comp'])
        
        max_x=max(x)
        min_x=min(x)
        max_z=max(z)
        min_z=min(z)
        max_y=max(y)
        min_y=min(y)
        message2='Atoms shown:'
        message2=message2+'\n'
        
        if self.i_rela.get()==0:M = float(self.atom_proc.get())   # M and N correspond to how many atoms are shown in the plot
        if self.i_rela.get()==1:M = float(self.atom_number.get())
        N=[]
        for i in range(0,len(u_ele)): 
            if self.i_rela.get()==1:
                e=label==u_ele[i]
                Total=len(self.tip['x'][e])
                Ns=int(math.ceil(Total/M))   
            if self.i_rela.get()==0:Ns=int(math.ceil(1/(M/100)))
            N.append(Ns)
        X_all=np.array([0,0,0])
        for j in range(0,len(index)):
            for i in range(0,len(u_ele)):                        # loop that goes over every different elements
                    if atoms[index][j]=='All': # if all was selected plot all of them together
                        en=label==u_ele[i] 
                        x_new=x[en]                                       # select the x values of each element indiviually
                        y_new=y[en]
                        z_new=z[en]
                        c_new=c[en]        
                        c_new[c_new=='#FFFFFF']=self.color_mapn[i]                  # if color of element is white change it to black (in order to see it)
                        X=np.array([x_new[::N[i]],y_new[::N[i]],z_new[::N[i]]]).T
                        X_all=np.vstack((X_all,X))
                        self.ax.scatter(x_new[::N[i]],y_new[::N[i]],z_new[::N[i]],c=c_new[::N[i]],label=u_ele[i],s=S,zorder = 1,depthshade = False,picker=True)  # scatter plot each element seperatly and assign label                                                 # assigne x_N value to show the number of plotted atoms in title
                        if u_ele[i]=='':message2=message2+'Unnamed'+str(i)+'  '+str(len(x_new[::N[i]]))
                        else:message2=message2+u_ele[i]+'  '+str(len(x_new[::N[i]]))
                        message2=message2+'\n'
                    elif atoms[index][j]==u_ele[i] and atoms[index][0]!='All':                                                 # if a certain element was selected in combobox for element selection
                        en=label==u_ele[i] 
                        x_new=x[en]                                       # select the x values of each element indiviually
                        y_new=y[en]
                        z_new=z[en]
                        c_new=c[en]  
                        X=np.array([x_new[::N[i]],y_new[::N[i]],z_new[::N[i]]]).T
                        X_all=np.vstack((X_all,X))
                        c_new[c_new=='#FFFFFF']=self.color_mapn[i]
                        self.ax.scatter(X[:,0],X[:,1],X[:,2],c=c_new[::N[i]],label=u_ele[i],s=S,zorder = 1, depthshade = False,picker=True)  #scatter plot each element seperatly and assign label         
                        if u_ele[i]=='':message2=message2+'Unnamed'+str(i)+u_ele[i]+'  '+str(len(X[:,0]))
                        else:message2=message2+u_ele[i]+'  '+str(len(X[:,0]))
                        message2=message2+'\n'
        X_all = np.delete(X_all, (0), axis=0)
        message2=message2+'Total:  '+str(len(X_all))
        set_axes_equal(self.ax)                                            # function for setting the axes equal for 3d plot
        if self.i_plot.get()==1:      self.ax.invert_zaxis()     # if inverse tip is selected inverse the z coordinates of the cylinder                                                        # show legends of each element
        if self.var_axis.get()==0:                                             # if show axis was selected
            self.ax.set_xlabel('X-axis',fontweight='bold')                     # label x axis
            self.ax.set_ylabel('Y-axis',fontweight='bold')                     # label y axis
            self.ax.set_zlabel('Z-axis',fontweight='bold')                     # label z axis
            self.ax.legend(loc='center left', bbox_to_anchor=(1.07, 0.5))#, fontsize=7)
        elif self.var_axis.get()==1:                                           # if show axis was not selected
            self.ax.set_axis_off()                                             # dont show axis
            
        if self.var_cyl.get()==1:                                              # if show clyinder was selected
            x_pos=self.cly_x.get()                                            # get x value from x entry box
            if x_pos=='': x_pos=0
            else:x_pos = float(x_pos)
            y_pos =self.cly_y.get()                                           # get y value from y entry box
            if y_pos=='': y_pos=0
            else:y_pos = float(y_pos)
            z_pos = self.cly_z.get()                                          # get z value from z entry box
            if z_pos=='': z_pos=0
            else:z_pos = float(z_pos) 
              
            height = float(self.height.get())                                 # get height value from height entry box
            alpha = float(self.alpha.get())                                   # get tilt along y value from alpha entry box
            beta = float(self.beta.get())                                     # get tilt along x value from beta entry box
            r = float(self.radius.get())                                      # get radius value from radius entry box 
            theta=np.linspace(0,2*np.pi,201)                                  # create linspace variable for plotting the circle
            alpha=math.radians(alpha)                                         # transform alpha from deg in rad
            beta=math.radians(beta)                                           # transfrom beta from deg in rad
            Xc,Yc,Zc=data_for_cylinder_along_z(0,0,0,r,height)  # calculating cylinder
            Z=np.array([Xc,Yc,Zc]).T
            matrix_l=np.array([(np.cos(beta),                   np.sin(alpha)*np.sin(beta),        np.cos(alpha)*np.sin(beta)),
                               (0,                              np.cos(alpha),                     -np.sin(alpha)),
                               (-np.sin(beta),                  np.sin(alpha)*np.cos(beta),        np.cos(alpha)*np.cos(beta))])
            pro=[]
            for j in range(0,len(Z)):
                 for i in range (0,len(Z[j])):
                         pro.append(matrix_l.dot(Z[i][j]))  
            pro=np.array(pro)                
            ZZ_new=pro.reshape(50,50,3).T
            self.ax.plot_surface(ZZ_new[0]+x_pos,ZZ_new[1]+y_pos,ZZ_new[2]+z_pos,alpha=0.4,color=color_zyl) # plotting cylinder
           
                
        if self.var_points.get()==1:
            XX=[]
            YY=[]
            ZZ=[]
            points=[]
            points=self.text_box.get(1.0,END)
            points=points.split('\n')
            points=[x for x in points if x!='']
            for i in range(0,len(points)):
                points_inner=points[i].split(',')
                xyz2=[]
                for j in points_inner:
                    m=j.split('=')
                    xyz2.append(m[1])
                x2=float(xyz2[0])
                y2=float(xyz2[1])
                z2=float(xyz2[2])
                XX.append(x2)
                YY.append(y2)
                ZZ.append(z2)
                self.ax.scatter3D(x2,y2,z2,marker='+',color=color_cp,s=S*100)
            if len(XX)>=3 and self.var_plane.get()==1:
                center_x=sum(XX)/len(XX)
                center_y=sum(YY)/len(YY)
                center_z=sum(ZZ)/len(ZZ)
                self.cly_x=StringVar(value=center_x)
                self.cly_y=StringVar(value=center_y)
                self.cly_z=StringVar(value=center_z)
                tmp_C=[]
                for i in range(len(XX)):
                    tmp_C.append([XX[i],YY[i],ZZ[i]])
                C=np.array(tmp_C).T
                svd=np.linalg.svd(C - np.mean(C, axis=1, keepdims=True))
                left=svd[0]
                fit=left[:,-1]
                normal=np.array([fit[0],fit[1],fit[2]])   
                r=10
                self.ax.plot([center_x,center_x+r*normal[0]],[center_y,center_y+r*normal[1]],[center_z,center_z+r*normal[2]],color='r',linewidth=5)
                xlim=self.ax.get_xlim()
                ylim=self.ax.get_ylim()
                X,Y=np.meshgrid(np.arange(xlim[0],xlim[1]),np.arange(ylim[0],ylim[1]))
                Z=np.zeros(X.shape)
                for r in range(X.shape[0]):
                    for c in range(X.shape[1]):
                        Z[r,c]=-(fit[0]*X[r,c]+fit[1]*Y[r,c])/fit[2]
                self.ax.plot_wireframe(X+center_x,Y+center_y,Z+center_z,color='k',alpha=.5)
        label2=np.array(self.tip['comp'])
        message='Atoms in specimen:'
        message=message+'\n'     
        for i in range(0,len(np.array(self.string))):
                if self.string[i]=='All':
                    numm=len(self.tip['x'])
                    NN=int(math.ceil(numm/M))
                else:
                    e=label2==u_ele[i-1]
                    numm=len(self.tip['x'][e])
                    NN=int(math.ceil(numm/M))
                message=message+self.string[i]+'  '+str(numm)
                message=message+'\n'
                
        self.message=Text(height = 15,width = 26,relief=FLAT)
        self.message.insert(END, message)
        self.message.configure(state='disabled')
        self.canvas_all.create_window(510,140,window=self.message,tags='message')
        self.message2=Text(height = 15,width = 26,relief=FLAT)
        self.message2.insert(END, message2)
        self.message2.configure(state='disabled')
        self.canvas_all.create_window(510,440,window=self.message2,tags='message')
        self.canvas.draw_idle()  
        self.plot_exist=1
  
        def distance(point, event):
#         Return distance between mouse position and given data point
#         Args:point (np.array): np.array of shape (3,), with x,y,z in data coords event (MouseEvent): mouse event (which contains mouse position in .x and .xdata)
#         Returns:distance (np.float64): distance (in screen coords) between mouse pos and data point          
             assert point.shape == (3,), "distance: point.shape is wrong: %s, must be (3,)" % point.shape  
             x2, y2, _ = proj3d.proj_transform(point[0], point[1], point[2], self.ax.get_proj())   # Project 3d data space to 2d data space
             x3, y3 = self.ax.transData.transform((x2, y2))                                        # Convert 2d data space to 2d screen space
             return np.sqrt ((x3 - event.x)**2 + (y3 - event.y)**2)                                # calculate distance
        
        def onclick(event):
                if event.key == "shift" and self.var_plane.get()==0:
                        distances = [distance (X_all[i,0:3], event) for i in range(X_all.shape[0])]                                              # use function distance to evaluate the closests index to mouse click
                        index=np.argmin(distances)                                                                                       # select the closest index
                        self.ax.scatter(X_all[index, 0], X_all[index, 1], X_all[index, 2],marker='+',color=color_cp,s=S*100)                         # plot controlpoint at closest index point
                        text='x='+str(np.round(X_all[index, 0],4))+', y='+str(np.round(X_all[index, 1],4))+', z='+str(np.round(X_all[index, 2],4))   # write x,y,z coordinates into text file
                        self.text_box.insert(END,text)                                                                                   # insert text file in textbox
                        self.text_box.insert(END,'\n')                                                                                   # make sure next control point is next row
                        self.canvas.draw_idle()  

        self.cid=self.fig.canvas.mpl_connect('button_press_event',onclick)
        self.fig.canvas.get_tk_widget().focus_force()
        
    def calc_con(self):                                                        # function for calculating the concentration
        if self.diss2==1:
            self.fig.canvas.mpl_disconnect(self.cid2) 
            self.diss2=0
        if self.plot_exist==1:
            self.fig.canvas.mpl_disconnect(self.cid) 
            self.plot_exist=0
        self.print_con=[]
        self.print_con2=[]
        self.canvas_all.delete('message') 
        start_end=self.cb_bulk.get()
        Z= self.cb_atom.curselection()                                                # getting variable from combobox of atom selection    
        index=np.array(Z)
        atom=np.array(self.string)
        u_ele_real=self.ele
        image_size=self.slider_image_size.get()
        image_size=float(image_size)/2
        self.fig.clear()
        self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[image_size,1],wspace=0.5,hspace=0.5,height_ratios=[image_size,1]) #adjust the size of the figure compared to (111)
        self.ax1=self.fig.add_subplot(self.spec[0])
        self.ax1.set_xlabel('z position of cylinder')
        self.ax1.set_ylabel('concentration [%]')
        x_real=np.array(self.tip['x'])
        y_real=np.array(self.tip['y'])
        z_real=np.array(self.tip['z'])
        c_real=np.array(self.tip['colour'])
        label_real=np.array(self.tip['comp'])
        inter=float(self.slider_inter.get()/20)
        height = float(self.height.get())
        alpha = float(self.alpha.get())
        beta = float(self.beta.get())
        r = float(self.radius.get())
        x_pos = float(self.cly_x.get())
        y_pos = float(self.cly_y.get())
        z_pos = float(self.cly_z.get())
        z_start=-height/2
        z_end=height/2
        alpha=math.radians(-alpha)
        beta=math.radians(-beta)
        ###################   tilt and move zylinder ######################################
        pro=[]
        Z=np.array([x_real-x_pos,y_real-y_pos,z_real-z_pos]).T
        M_ROT1=[(1,      0,                  0     ),            # rotational matrix along the x-direction
                (0,  np.cos(alpha),  -np.sin(alpha)),
                (0,  np.sin(alpha),    np.cos(alpha))]

        M_ROT2=[(np.cos(beta),      0,       np.sin(beta)),     # rotational matrix along the y-direction
                (0,                  1,            0      ),
                (-np.sin(beta),      0,       np.cos(beta))]
        matrix_l=np.array(M_ROT1).dot(M_ROT2)
        for j in range(0,len(Z)):
                    pro.append(matrix_l.dot(Z[j])) 
        Z_new=np.array(pro)
        x_real=Z_new[:,0]
        y_real=Z_new[:,1]
        z_real=Z_new[:,2]
        ######################################################################### 
        xy=((x_real)**2+(y_real)**2)**(1/2)                                    # cirlce constraint for atoms       
        circle=xy<=r                                                           # only select atoms inside of circle
        message='concentration: \n'                                            # initialize the message displayed on the right side of the plot
        for i in range(0,len(index)):
              for m in range(0,len(u_ele_real)):                                      # loop over each individual elements
                if atom[index][i]=='All':                                                    # seperate if atom select checkbox 'all' is selected
                    plot_z=[]                                                          # x component of concentration plots 
                    Num=[]                                                             # Number of atoms inside the circle and intervals (for concentration calculation)
                    Num_all=[]                                                         # total number of atoms (of each element) to calculate the % contribution of each
                    e=label_real[circle]==u_ele_real[m]                                # seperation of the elements
                    z_circle=z_real[circle]                                            # only selec the atoms with circle constraint
                    z_p=z_circle[e]                                                    # only select the atoms wth circle constraint of specific element
                    c_circle=c_real[circle]
                    c_p=c_circle[e]
                    for j in range (0,int((z_end-z_start)/inter)+1):                     # loop of intervals
                        init=z_p>=z_start+j*inter                                      # starting condition of interval for each element
                        init_all=z_circle>=z_start+j*inter                             # starting condition of interval for all elements together
                        z_all2=z_circle[init_all]                                      # selecting atoms which fulfill starting constraint for all elements
                        z_p2=z_p[init]                                                 # selecting atoms which fulfill the starting constraint for each individual element
                        init2=z_p2<=z_start+(j*inter+inter)                            # ending constraint of interval
                        init2_all=z_all2<=z_start+(j*inter+inter)                      # ending constraint of interval for all elements
                        z_all3=z_all2[init2_all]                                       # selecting atoms which also fulfill ending constraint for all elements
                        z_p3=z_p2[init2]                                               # selecting atom which also fulfill ending constraint for each individual elements
                        Num_all.append(len(z_all3))                                    # append the number of atoms which fulfill both contraints
                        Num.append(len(z_p3))                                          # append the number of atoms which fulfill both constraints for each individual element
                        plot_z.append(z_start+j*inter)#+inter/2)                       # append the x-coordinate for the concentration plot
                    a=np.array(Num,dtype=float)                                        # transform total number of atoms for each individual element into array of floats
                    b=np.array(Num_all,dtype=float)                                    # transform total number of atoms into array of floats
                    if self.var_switch.get()==1: 
                        if start_end=='start':b=np.array(Num_all[0])
                        last=len(np.array(Num_all))-1    
                        if start_end=='end':b=np.array(Num_all[last])
                        if b==0:messagebox.showinfo(title='APTyzer',message='Reference bulk concentration is 0')
                    self.conc=100*np.divide(a,b, out=np.zeros_like(a), where=b!=0)     # calculate concentration, if total amount of atoms == 0, result is 0
                    message=message+u_ele_real[m]                                      # append element type into message
                    message=message+': maximal value: '                                # add text to message
                    message=message+str(round(max(self.conc),2))                            # add maximum of concentration to message
                    message=message+' minimal value: '                                 # add text to message
                    message=message+str(round(min(self.conc),2))                            # add minimum of concentration to message
                    message=message+'\n'                                               # add line split to message                                               
                    if c_p[0]=='#FFFFFF':c_p[0]=self.color_mapn[m]    # if color of particles is white change it to black so it can be seen with white background
                    self.ax1.plot(plot_z,self.conc,c_p[0],label=u_ele_real[m])                 # plot the concentration of each element individually
                    self.print_con2.append(u_ele_real[m])
                    self.print_con.append(self.conc)
                elif u_ele_real[m]==atom[index][i] and atom[index][0]!='All':          # if certain element is selected in checkbox
                    plot_z=[]                                                          # x component of concentration plots 
                    Num=[]                                                             # Number of atoms inside the circle and intervals (for concentration calculation)
                    Num_all=[]                                                         # total number of atoms (of each element) to calculate the % contribution of each
                    e=label_real[circle]==u_ele_real[m]                                # seperation of the elements
                    z_circle=z_real[circle]                                            # only selec the atoms with circle constraint
                    z_p=z_circle[e]                                                    # only select the atoms wth circle constraint of specific element
                    c_circle=c_real[circle]
                    c_p=c_circle[e]
                    for j in range (0,int((z_end-z_start)/inter)+1):                     # loop of intervals
                        init=z_p>=z_start+j*inter                                      # starting condition of interval for each element
                        init_all=z_circle>=z_start+j*inter                             # starting condition of interval for all elements together
                        z_all2=z_circle[init_all]                                      # selecting atoms which fulfill starting constraint for all elements
                        z_p2=z_p[init]                                                 # selecting atoms which fulfill the starting constraint for each individual element
                        init2=z_p2<=z_start+(j*inter+inter)                            # ending constraint of interval
                        init2_all=z_all2<=z_start+(j*inter+inter)                      # ending constraint of interval for all elements
                        z_all3=z_all2[init2_all]                                       # selecting atoms which also fulfill ending constraint for all elements
                        z_p3=z_p2[init2]                                               # selecting atom which also fulfill ending constraint for each individual elements
                        Num_all.append(len(z_all3))                                    # append the number of atoms which fulfill both contraints
                        Num.append(len(z_p3))                                          # append the number of atoms which fulfill both constraints for each individual element
                        plot_z.append(z_start+j*inter)#+inter/2)                       # append the x-coordinate for the concentration plot
                    a=np.array(Num,dtype=float)                                        # transform total number of atoms for each individual element into array of floats
                    b=np.array(Num_all,dtype=float)                                    # transform total number of atoms into array of floats
                    if self.var_switch.get()==1: 
                        if start_end=='start':b=np.array(Num_all[0])
                        last=len(np.array(Num_all))-1    
                        if start_end=='end':b=np.array(Num_all[last])
                        if b==0:messagebox.showinfo(title='APTyzer',message='Reference bulk concentration is 0')
                    self.conc=100*np.divide(a,b, out=np.zeros_like(a), where=b!=0)     # calculate concentration, if total amount of atoms == 0, result is 0
                    message=message+u_ele_real[m]                                      # append element type into message
                    message=message+': maximal value: '                                # add text to message
                    message=message+str(round(max(self.conc),2))                            # add maximum of concentration to message
                    message=message+' minimal value: '                                 # add text to message
                    message=message+str(round(min(self.conc),2))                            # add minimum of concentration to message
                    message=message+'\n'                                               # add line split to message  
                    if c_p[0]=='#FFFFFF':c_p[0]=self.color_mapn[m]             # if color of particles is white change it to black so it can be seen with white background
                    self.ax1.plot(plot_z,self.conc,c_p[0],label=u_ele_real[m])                 # plot the concentration of each element individually
                    self.print_con2.append(u_ele_real[m])
                    self.print_con.append(self.conc)
        self.print_con2.append('z_cyl')
        plot_z2=plot_z.copy()
        plot_z2=np.array(plot_z2).reshape(np.shape(np.array(self.conc)))
        self.print_con.append(plot_z2)
        self.df_con=pd.DataFrame(np.array(self.print_con).T,columns=self.print_con2)
        self.ax1.legend(loc='center left', bbox_to_anchor=(1.02, 0.5))
        self.ax1.set_title('concentration of elements along cylinder / intervalsize=%1.2f' %inter)
        self.canvas.draw_idle()
        self.message=Text(height = 8,width = 50,relief=FLAT)
        self.message.insert(END, message)
        self.message.configure(state='disabled')
        self.canvas_all.create_window(200,770,window=self.message,tags='message')
        self.conc_check=1
        
        
        
    def calc_excess(self):
        if self.diss2==1:
            self.fig.canvas.mpl_disconnect(self.cid2) 
            self.diss2==0
        if self.plot_exist==1:
            self.fig.canvas.mpl_disconnect(self.cid) 
            self.plot_exist=0
        self.print_ex=[]
        self.print_ex2=[]
        self.canvas_all.delete('message')  
        Z= self.cb_atom.curselection()                                                # getting variable from combobox of atom selection    
        index=np.array(Z)
        atom=np.array(self.string)      
        color_zyl=str(self.cb_color.get())
        u_ele_real=self.ele        
        image_size=self.slider_image_size.get()
        image_size=float(image_size)/2
        self.fig.clear()
        self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[image_size,1],wspace=0.5,hspace=0.5,height_ratios=[image_size,1]) #adjust the size of the figure compared to (111)
        self.ax1=self.fig.add_subplot(self.spec[0])
        self.ax1.set_xlabel('z position of cylinder')
        self.ax1.set_ylabel('Total amount of atoms')
        x_real=np.array(self.tip['x'])
        y_real=np.array(self.tip['y'])
        z_real=np.array(self.tip['z'])                             # atoms z coordinatines inversion
        c_real=np.array(self.tip['colour'])
        label_real=np.array(self.tip['comp'])
        inter=float(self.slider_inter.get()/20)
        height = float(self.height.get())
        alpha = float(self.alpha.get())
        beta = float(self.beta.get())
        r = float(self.radius.get())
        x_pos = float(self.cly_x.get())
        y_pos = float(self.cly_y.get())
        z_pos = float(self.cly_z.get())
        z_start=-height/2
        z_end=height/2
        alpha=math.radians(-alpha)                               # transfroms radius into radians
        beta=math.radians(-beta)                                 # - because otherwise it tilts the cylinder in the wrong direction (dont know why)
        ###################   tilt and move zylinder ######################################
        pro=[]
        Z=np.array([x_real-x_pos,y_real-y_pos,z_real-z_pos]).T
        M_ROT1=[(1,      0,                  0     ),            # rotational matrix along the x-direction
                (0,  np.cos(alpha),  -np.sin(alpha)),
                (0,  np.sin(alpha),    np.cos(alpha))]

        M_ROT2=[(np.cos(beta),      0,       np.sin(beta)),     # rotational matrix along the y-direction
                (0,                  1,            0      ),
                (-np.sin(beta),      0,       np.cos(beta))]
        matrix_l=np.array(M_ROT1).dot(M_ROT2)
        for j in range(0,len(Z)):
                    pro.append(matrix_l.dot(Z[j])) 
        Z_new=np.array(pro)
        x_real=Z_new[:,0]
        y_real=Z_new[:,1]
        z_real=Z_new[:,2]
        ######################################################################### 
        xy=((x_real)**2+(y_real)**2)**(1/2)
        circle=xy<=r
        number_of_elements=0
        for i in range(0,len(index)):
            for m in range(0,len(u_ele_real)):
                if atom[index][i]=='All':
                    plot_z=[]
                    excess=[]
                    ex=0
                    e=label_real[circle]==u_ele_real[m]               #seperation of the atoms 
                    z_circle=z_real[circle]
                    z_p=z_circle[e]
                    c_circle=c_real[circle]
                    c_p=c_circle[e]
                    for j in range (0,int((z_end-z_start)/inter)+1):
                        init=z_p>=z_start+j*inter   
                        z_p2=z_p[init]
                        init2=z_p2<=z_start+(j*inter+inter)
                        z_p3=z_p2[init2]
                        ex+=len(z_p3)
                        excess.append(ex)
                        plot_z.append(z_start+j*inter)#+inter/2)   
                    if self.var_norm.get()==1:excess=np.array(excess)/(max(excess))
                    if c_p[0]=='#FFFFFF':c_p[0]=self.color_mapn[m]
                    self.ax1.plot(plot_z,excess,c_p[0],label=u_ele_real[m])
                    excess_save=excess.copy()
                    self.print_ex2.append(u_ele_real[m])
                    self.print_ex.append(np.array(excess))
                    number_of_elements=number_of_elements+1
                elif u_ele_real[m]==atom[index][i] and atom[index][0]!='All':
                    plot_z=[]
                    excess=[]
                    ex=0
                    e=label_real[circle]==u_ele_real[m]               #seperation of the atoms 
                    z_circle=z_real[circle]
                    z_p=z_circle[e]
                    c_circle=c_real[circle]
                    c_p=c_circle[e]
                    for j in range (0,int((z_end-z_start)/inter)+1):
                        init=z_p>=z_start+j*inter   
                        z_p2=z_p[init]
                        init2=z_p2<=z_start+(j*inter+inter)
                        z_p3=z_p2[init2]
                        ex+=len(z_p3)
                        excess.append(ex)
                        plot_z.append(z_start+j*inter)#+inter/2)   
                    if self.var_norm.get()==1:excess=np.array(excess)/(max(excess))
                    if c_p[0]=='#FFFFFF':c_p[0]=self.color_mapn[m]
                    self.ax1.plot(plot_z,excess,c_p[0],label=u_ele_real[m]) 
                    excess_save=excess.copy()
                    self.print_ex2.append(u_ele_real[m])
                    self.print_ex.append(np.array(excess)) 
                    number_of_elements=number_of_elements+1
        self.print_ex2.append('z_cyl')
        plot_z2=plot_z.copy()
        plot_z2=np.array(plot_z2).reshape(np.shape(np.array(excess)))
        self.print_ex.append(plot_z2)
        self.df_ex=pd.DataFrame(np.array(self.print_ex).T,columns=self.print_ex2)
        self.ax1.set_title('Excess (total atom number of elements along cylinder) / intervalsize=%1.2f ' %inter)
        self.ax1.legend(loc='center left', bbox_to_anchor=(1.07, 0.5))
        
        points_ex=self.text_box_ex.get(1.0,END)   # get text from text box
        if points_ex!='\n':
            points_ex=points_ex.split('\n')           # split the text into each row
            points_ex=[x for x in points_ex if x!=''] # remove any rows that dont contain information
            XX=[]
            YY=[]
            for i in range(0,len(points_ex)):      # loop over all rows
                    points_inner=points_ex[i].split(',')   # split each row into parts of each coordinates
                    xy2=[]
                    for j in points_inner:               # loop over all values inside row
                        m=j.split('=')                   # isolate actual number
                        xy2.append(m[1])                # save number
                    XX.append(float(xy2[0]))            # save x,y and z coordinate
                    YY.append(float(xy2[1]))
            if len(XX)==4:
                if number_of_elements>1:messagebox.showinfo(title='APTyzer',message='For optimal excess calculation only show 1 atom type.')
                k1=(YY[0]-YY[1])/(XX[0]-XX[1])
                k3=(YY[2]-YY[3])/(XX[2]-XX[3])
                d1=YY[1]-k1*XX[1]
                d3=YY[3]-k3*XX[3]
                x=np.array([-height/2,height/2])
                y1=gerade(np.array(k1),x,np.array(d1))
                y3=gerade(np.array(k3),x,np.array(d3))
                diff=[]
                for i in range(0,len(excess_save)):
                    upper_line=gerade(np.array(k3),plot_z[i],np.array(d3))
                    lower_line=gerade(np.array(k1),plot_z[i],np.array(d1))  
                    diff.append(np.abs((upper_line-excess_save[i])-(excess_save[i]-lower_line)))
                plot_z=np.array(plot_z)
                x_found=plot_z[diff==min(diff)]
                y_excess_u=gerade(np.array(k3),x_found,np.array(d3))
                y_excess_l=gerade(np.array(k1),x_found,np.array(d1))
                excess_atom_value=y_excess_u[0]-y_excess_l[0]
                excess_value=excess_atom_value/(np.pi*r**2)
                self.ax1.plot([x_found,x_found],[y_excess_l,y_excess_u])
                self.ax1.plot(x,y1,color=color_zyl)
                self.ax1.plot(x,y3,color=color_zyl)
                self.ax1.scatter(XX,YY,color='black',marker='x')
                self.ax1.annotate(" 1", (XX[0], YY[0]))
                self.ax1.annotate(" 2", (XX[1], YY[1]))
                self.ax1.annotate(" 3", (XX[2], YY[2]))
                self.ax1.annotate(" 4", (XX[3], YY[3]))
                self.output = Text(height = 4,width = 40,relief=FLAT)
                self.canvas_all.create_window(200,750,window=self.output,tags='message')
                message2='Results: \n'
                message2=message2+'the interfacial excess atoms = '+str(np.round(excess_atom_value,3))+' atoms'+'\n'
                message2=message2+'the interfacial excess = '+str(np.round(excess_value,3))+' atoms/nm²'
                self.output.insert(END, message2)
                self.output.configure(state='disabled')
                self.canvas.draw_idle()    
            elif len(XX)>=4:messagebox.showinfo(title='APTyzer',message='Too many fit points selected. Reduce to 4 points to fit excess.')
        def onclick2(event):
            if event.key == "shift":
                x=np.round(event.xdata,2)
                y=np.round(event.ydata,2)
                self.ax1.scatter(x,y,color='black',marker='x')
                text='x='+str(x)+', y='+str(y)
                self.text_box_ex.insert(END,text)
                self.text_box_ex.insert(END,'\n')
                self.canvas.draw_idle()   
        if self.diss2==1:self.fig.canvas.mpl_disconnect(self.cid2)    
        self.cid2=self.fig.canvas.mpl_connect('button_press_event',onclick2)
        self.diss2=1
        self.canvas.draw_idle()  
        self.excess_check=1
        self.fig.canvas.get_tk_widget().focus_force()
                    
    def calc_zoom(self):     
        if self.plot_exist==1:  # if plot already exists get the parameters of old plot
            self.fig.canvas.mpl_disconnect(self.cid)
            self.plot_exist=0
        if self.i_plot.get()==0:self.invert=0                                 # if inversion checkbox is unmarked set value=0
        else: self.invert=1                                                   # if inversion checkbox is marked set value=1
        if self.diss2==1:
            self.fig.canvas.mpl_disconnect(self.cid2) 
            self.diss2=0
        self.fig.clear()                                                   # if plot single was selected clear all previous plots
        image_size=self.slider_image_size.get()
        image_size=float(image_size)**2
        self.spec=gridspec.GridSpec(ncols=2,nrows=2,width_ratios=[image_size,1],wspace=0.1,hspace=0.1,height_ratios=[image_size,1]) #adjust the size of the figure compared to (111)
        self.ax2=self.fig.add_subplot(self.spec[0],projection='3d')         # make new figure with size depending on the image size slider    
        color_zyl=str(self.cb_color.get())                                    # get color of cylinder from color entry box
        color_cp=str(self.cb_color_cp.get())
        self.canvas_all.delete('message')                                      # clearing message about concentration data

        Zp= self.cb_atom.curselection()                                         # getting variable from listbox of atom selection    
        if Zp==(): 
            self.cb_atom.select_set(0) 
            Zp= self.cb_atom.curselection() 
        index=np.array(Zp)
        atoms=np.array(self.string)
        #M = self.slider_res.get()                                              # getting variable from combobox of resolution selection
        M = float(self.atom_number.get())
        SS=self.slider_size.get()                                              # getting variable from combobox of atom size selection
        S=float(SS)/100                                                        # transforming entry into float and deviding by 10 so values 0.1 to 1 are also included
        image_size=self.slider_image_size.get()
        image_size=float(image_size)**2
        u_ele_real=self.ele

        x_real=np.array(self.tip['x'])
        y_real=np.array(self.tip['y'])    
        z_real=np.array(self.tip['z'])
        c_real=np.array(self.tip['colour'])
        label_real=np.array(self.tip['comp'])     
        height = float(self.height.get())
        alpha = -float(self.alpha.get())
        beta = -float(self.beta.get())
        r = float(self.radius.get())
        x_pos = float(self.cly_x.get())
        y_pos = float(self.cly_y.get())
        z_pos = float(self.cly_z.get())
        color_zyl=str(self.cb_color.get())
        z_start=-height/2
        z_end=height/2
        alpha=math.radians(alpha)
        beta=math.radians(beta)
        ###################   tilt and move zylinder ######################################
        pro=[]
        Z=np.array([x_real-x_pos,y_real-y_pos,z_real-z_pos]).T
        M_ROT1=[(1,      0,                  0     ),            # rotational matrix along the x-direction
                (0,  np.cos(alpha),  -np.sin(alpha)),
                (0,  np.sin(alpha),    np.cos(alpha))]

        M_ROT2=[(np.cos(beta),      0,       np.sin(beta)),     # rotational matrix along the y-direction
                (0,                  1,            0      ),
                (-np.sin(beta),      0,       np.cos(beta))]
        matrix_l=np.array(M_ROT1).dot(M_ROT2)
        for j in range(0,len(Z)):
                    pro.append(matrix_l.dot(Z[j])) 
        Z_new=np.array(pro)
        x_real=Z_new[:,0]
        y_real=Z_new[:,1]
        z_real=Z_new[:,2]
        #########################################################################   
        xy=((x_real)**2+(y_real)**2)**(1/2)
        circle=xy<=r
        message='Atoms in ROI:\n'
        message2='Shown:\n'
#         N=[]        
#         for i in range(0,len(u_ele_real)):
#             e=label_real==u_ele_real[i]
#             Total=len(self.tip['x'][e])
#             Ns=int(math.ceil(Total/M))
#             if Ns==0:Ns=1
#             N.append(Ns)
        for i in range(0,len(index)):
            for m in range(0,len(u_ele_real)):
                if atoms[index][i]=='All':
                    e=label_real[circle]==u_ele_real[m]
                    z_circle=z_real[circle]
                    x_circle=x_real[circle]
                    y_circle=y_real[circle]
                    c_circle=c_real[circle]
                    z_p=z_circle[e]  
                    x_p=x_circle[e] 
                    y_p=y_circle[e]
                    c_p=c_circle[e]
                    init_plot=z_p>=z_start
                    x_pl2=x_p[init_plot]
                    y_pl2=y_p[init_plot]
                    z_pl2=z_p[init_plot]
                    c_pl2=c_p[init_plot]
                    init_plot2=z_pl2<=z_end    
                    x_pl3=x_pl2[init_plot2]
                    y_pl3=y_pl2[init_plot2]
                    z_pl3=z_pl2[init_plot2]
                    c_pl3=c_pl2[init_plot2]
                    x_N=len(x_pl3)
                    N=int(math.ceil(x_N/M))
                    if N==0:N=1
                    x_M=len(x_pl3[::N])
                    message=message+str(u_ele_real[m])
                    message=message+' '+str(x_N)+'\n'
                    message2=message2+str(u_ele_real[m])
                    message2=message2+' '+str(x_M)+'\n'
                    c_pl3[c_pl3=='#FFFFFF']=self.color_mapn[i] 
                    self.ax2.scatter3D(x_pl3[::N],y_pl3[::N],z_pl3[::N],c=c_pl3[::N],label=u_ele_real[m],s=S)
                elif u_ele_real[m]==atoms[index][i] and atoms[index][0]!='All':
                    z_circle=z_real[circle]
                    x_circle=x_real[circle]
                    y_circle=y_real[circle]
                    c_circle=c_real[circle]
                    e=label_real[circle]==u_ele_real[m] 
                    z_p=z_circle[e]  
                    x_p=x_circle[e] 
                    y_p=y_circle[e]
                    c_p=c_circle[e]
                    init_plot=z_p>=z_start
                    x_pl2=x_p[init_plot]
                    y_pl2=y_p[init_plot]
                    z_pl2=z_p[init_plot]
                    c_pl2=c_p[init_plot]
                    init_plot2=z_pl2<=z_end    
                    x_pl3=x_pl2[init_plot2]
                    y_pl3=y_pl2[init_plot2]
                    z_pl3=z_pl2[init_plot2]
                    c_pl3=c_pl2[init_plot2]
                    c_pl3[c_pl3=='#FFFFFF']=self.color_mapn[i] 
                    x_N=len(x_pl3)
                    N=int(math.ceil(x_N/M))
                    if N==0:N=1
                    x_M=len(x_pl3[::N])
                    message=message+str(u_ele_real[m])
                    message=message+' '+str(x_N)+'\n'
                    message2=message2+str(u_ele_real[m])
                    message2=message2+' '+str(x_M)+'\n'
                    self.ax2.scatter3D(x_pl3[::N],y_pl3[::N],z_pl3[::N],c=c_pl3[::N],label=u_ele_real[m],s=S)                 
        if self.var_axis.get()==0: 
            self.ax2.set_xlabel('X-axis',fontweight='bold')
            self.ax2.set_ylabel('Y-axis',fontweight='bold')
            self.ax2.set_zlabel('Z-axis',fontweight='bold')
            self.ax2.legend()
        elif self.var_axis.get()==1:
            self.ax2.set_axis_off()
        set_axes_equal(self.ax2)   
        if self.var_cyl.get()==1:
            x_pos = float(self.cly_x.get())
            y_pos = float(self.cly_y.get())
            z_pos = float(self.cly_z.get())
            height = float(self.height.get())
            alpha = float(self.alpha.get())
            beta = float(self.beta.get())
            r = float(self.radius.get())
            color_zyl=str(self.cb_color.get())
            theta=np.linspace(0,2*np.pi,201)
            z_start=-height/2
            z_end=height/2
            alpha=math.radians(alpha)
            beta=math.radians(beta)
        
            y_circle=r*np.cos(theta)
            x_circle=r*np.sin(theta)
            z_circle_s=np.ones(201)*z_start
            z_circle_e=np.ones(201)*z_end

            x_line1=np.ones(201)*r
            x_line2=np.ones(201)*-r
            y_line=np.ones(201)*0
            z_line=np.linspace(z_start,z_end,201)
            lw=5
            self.ax2.plot(x_line1,y_line,z_line,color_zyl,linewidth=lw)
            self.ax2.plot(x_line2,y_line,z_line,color_zyl,linewidth=lw)
            self.ax2.plot(x_circle,y_circle,z_circle_s,color_zyl,linewidth=lw)  
            self.ax2.plot(x_circle,y_circle,z_circle_e,color_zyl,linewidth=lw) 
        self.message=Text(height = 15,width = 26,relief=FLAT)
        self.message.insert(END, message)
        self.message.configure(state='disabled')
        self.canvas_all.create_window(520,140,window=self.message,tags='message')
        self.message2=Text(height = 15,width = 26,relief=FLAT)
        self.message2.insert(END, message2)
        self.message2.configure(state='disabled')
        self.canvas_all.create_window(520,440,window=self.message2,tags='message')
        self.canvas.draw_idle() 

tk = Tk()                                                                      # tkinter
tk.title("APTyzer")                                                       # select title of tool
tk.geometry('1100x700')                                                        # select size of tool
tt=APTyzer(tk)                                                            # select tool              
tk.mainloop()                                                                  # start tool